# Оценка функции времени работы для BattleField

_Дано_: количество завершенных матчей $things$, количество потоков $jobs$.

_Требуется_: оценить матожидание общего времени работы при условии, что матожидание времени одного матча $\textbf{E} \xi$ равно $1.0$. Распределение $\xi$ неизвестно, поэтому для простоты предполагаем, что $\xi\sim \mathcal{N}(1.0, 0.15)$.

Для оценки будем использовать функцию $f(jobs, things) = \frac{jobs}{things} + G(jobs, things)$, где $G(jobs, things)$ &mdash; некоторая модель, которую мы ниже и будем обучать. Я люблю линейные модели, поэтому модель будет линейной регрессией с наинжиниренными фичами.

Оценивать качество будем так. Пусть $y_1, y_2, \dots, y_n$ &mdash; истинные значения матожидания (полученные с помощью метода Монте-Карло), а $y'_1, y'_2, \dots, y'_n$ &mdash; наши оценки. Тогда для оценки моделей рассмотрим скоры $s_{avg}$ и $s_{max}$, которые вычисляются по формулам:
$$s_{avg} = \sqrt{\frac{\sum_\limits{i=1}^n \left(\frac{y_i - y'_i}{y_i}\right)^2 }n}$$
$$s_{max} = \max\limits_{i=1\dots n}{\left| \frac{y_i - y'_i}{y_i} \right|}$$

## Генерация датасета

Сначала напишем функцию для симуляции процесса с помощью метода Монте-Карло:

In [168]:
import heapq

def simulate(jobs, things, sigma=0.15, iters=10_000):
    sum_time = 0.0

    for i in range(iters):
        heap = []
        time = 0.0
        for _ in range(things):
            if len(heap) == jobs:
                time = heapq.heappop(heap)
            heapq.heappush(heap, time + max(0.0, random.gauss(1.0, sigma)))
        time = max(heap + [time])
        sum_time += time

    return sum_time / iters

Сгенерируем датасет. Будем генерировать его честно, чтобы строчки с маленькими значениями $jobs$ и $things$ тоже встречались достаточно большое количество раз.

In [169]:
import random
import copy

def expand(my_list, need_size):
    if len(my_list) >= need_size:
        return copy.copy(my_list)
    new_list = copy.copy(my_list)
    for i in range(len(my_list), need_size):
        new_list.append(random.choice(my_list))
    return new_list
    
dataset = []
for jobs in range(1, 33):
    small = random.sample(range(1, 4 * jobs + 1), k=min(4*jobs, 20))
    small = expand(small, 20)
    large = random.sample(range(4 * jobs + 1, 12 * jobs + 1), k=min(4*jobs, 15))
    large = expand(large, 15)
    dataset += [[jobs, things] for things in small + large]

X = np.array(dataset).astype(dtype=np.float64)
X

Сгенерируем ответы (осторожно, это достаточно долго!):

In [ ]:
from tqdm.notebook import tqdm

y = [simulate(int(ln[0]), int(ln[1])) - ln[1] / ln[0] for ln in tqdm(X)]
y

Сохраним датасет, чтобы не потерять его. А то он довольно долго генерируется :(

In [178]:
header = 'jobs,things,y'
dataset = np.c_[X, np.array(y).reshape(-1, 1)]
np.savetxt('dataset1.csv', dataset, fmt='%.14f', delimiter=',', header=header, comments='')

## Обучение

Загружаем датасет:

In [356]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('dataset0.csv').values
X = dataset[:, :-1]
y = np.array(dataset[:, -1])

Расширим датасет дополнительными признаками: $1$, $\log things$, $\frac 1{jobs}$, $\frac{\log things}{jobs}$, $\frac 1{jobs^2}$, $\frac {\log things}{jobs^2}$, $\frac {things}{jobs^2}$:

In [357]:
jobs = X[:, 0]
things = X[:, 1]
X = np.c_[
    jobs,
    things,
    np.array([1. for _ in X]),
    np.log(things),
    1. / jobs,
    np.log(things) / jobs,
    1. / (jobs ** 2),
    np.log(things) / (jobs ** 2),
    things / (jobs ** 2),
]

Делим наш датасет на `train` и `test`:

In [358]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Вводим метрики (точные формулы см. выше):

In [364]:
def estimate(X, y_pred, y_real):
    y1_pred = y_pred + X[:, 1] / X[:, 0]
    y1_real = y_real + X[:, 1] / X[:, 0]
    return np.sqrt(np.mean((y1_real / y1_pred - 1.) ** 2))

def estimate_max(X, y_pred, y_real):
    y1_pred = y_pred + X[:, 1] / X[:, 0]
    y1_real = y_real + X[:, 1] / X[:, 0]
    return np.max(np.abs(y1_real / y1_pred - 1.))

Генерим константные предсказания, чтобы понять baseline:

In [365]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor()
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)
print('Average:', estimate(X_test, y_pred, y_test))
print('Max:', estimate_max(X_test, y_pred, y_test))

Average: 0.10398750297737731
Max: 0.5605603872498814


Обучаем линейную регрессию:

In [366]:
from sklearn.linear_model import Ridge, Lasso

model = Lasso(alpha=1e-5, max_iter=10000, fit_intercept=False)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Average:', estimate(X_test, y_pred, y_test))
print('Max:', estimate_max(X_test, y_pred, y_test))

Average: 0.05484311003621325
Max: 0.23374952098685364


А если на всех данных обучить? (Скоры ниже получены на той же выборке, на которой обучалась модель, поэтому могут быть необъективны)

In [367]:
from sklearn.base import clone

model = clone(model)
model.fit(X, y)
y_pred = model.predict(X)
print('Average:', estimate(X, y_pred, y))
print('Max:', estimate_max(X, y_pred, y))

Average: 0.05800443672073826
Max: 0.2402331194843712


Взглянем на коэффициенты, чтобы применить результат обучения в самой программе:

In [368]:
model.coef_

array([ 2.77918644e-03,  8.64226651e-04,  9.46858137e-01, -1.07427060e-01,
       -1.38769432e+00,  2.61388592e-02,  4.51703118e-01, -1.17767833e-01,
        4.56262770e-02])